In [1]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn as nn

In [2]:
from huggingface_hub import login

#Enter your token here
HF_TOKEN = ''

login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
baichuan_path = 'baichuan-inc/Baichuan-7B'
baichuan_model = AutoModelForCausalLM.from_pretrained(baichuan_path, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

configuration_baichuan.py:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- configuration_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_baichuan.py:   0%|          | 0.00/28.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- modeling_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/14.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(baichuan_path, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

tokenization_baichuan.py:   0%|          | 0.00/9.58k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baichuan-inc/Baichuan-7B:
- tokenization_baichuan.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [5]:
INTERVAL = 2
MERGE_LAYERS = 4
HIGHEST_LAY = 32
LOWEST_LAY = 1
THRESHOLD = 0.7

In [6]:
from copy import deepcopy
def merge_layers_return_model(model, merge_base_lay, merge_layer_num):
    merge_layer_num = min(merge_layer_num, len(model.model.layers) - merge_base_lay - 1)

    model_copy = deepcopy(model)
    for diff_lay in range(merge_base_lay+1, merge_base_lay+1+merge_layer_num):
        # gate_proj
        model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.gate_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.gate_proj.weight.data
        )
        # down_proj
        model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.down_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.down_proj.weight.data
        )
        # up_proj
        model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data.add_(
            model.model.layers[diff_lay].mlp.up_proj.weight.data - model_copy.model.layers[merge_base_lay].mlp.up_proj.weight.data
        )
        # W_pack
        model_copy.model.layers[merge_base_lay].self_attn.W_pack.weight.data.add_(
            model.model.layers[diff_lay].self_attn.W_pack.weight.data - model_copy.model.layers[merge_base_lay].self_attn.W_pack.weight.data
        )
        # o_proj
        model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data.add_(
            model.model.layers[diff_lay].self_attn.o_proj.weight.data - model_copy.model.layers[merge_base_lay].self_attn.o_proj.weight.data
        )
    for diff_lay in range(merge_base_lay+merge_layer_num, merge_base_lay, -1):
        del(model_copy.model.layers[diff_lay])
    return model_copy

In [7]:
import copy
baichuan_copy_to_compress = copy.deepcopy(baichuan_model)

In [8]:
import numpy as np
def cal_last_hidden_sim(model1, model2, tokenizer, sents):
    sim_ls = []
    for s in sents:
        encoded_inputs = tokenizer(s, return_tensors='pt')
        with torch.no_grad():
            # Disable cache during forward pass
            outputs1 = model1(**encoded_inputs, output_hidden_states=True, use_cache=False)
            hidden_states1 = outputs1.hidden_states[-1]
        with torch.no_grad():
            outputs2 = model2(**encoded_inputs, output_hidden_states=True, use_cache=False)
            hidden_states2 = outputs2.hidden_states[-1]

        similarity = torch.cosine_similarity(
            hidden_states1.squeeze(0).flatten().unsqueeze(0),
            hidden_states2.squeeze(0).flatten().unsqueeze(0)
        )
        sim_ls.append(similarity)

    sim_ls = [i.item() for i in sim_ls]
    print("Similarities:", sim_ls, "Mean similarity:", np.mean(sim_ls))
    return np.mean(sim_ls)



In [9]:
lay = HIGHEST_LAY - MERGE_LAYERS
last_merge_flag = False

sents= ['Torreorgaz is a municipality in the', 'The 81st Mechanised Brigade () is a']

while lay >= LOWEST_LAY:
    print(lay)
    print('current model layer', len(baichuan_copy_to_compress.model.layers))
    tmp_merged_model = merge_layers_return_model(baichuan_copy_to_compress, lay, MERGE_LAYERS-1)
    sim_value = cal_last_hidden_sim(baichuan_model, tmp_merged_model, tokenizer, sents)
    if sim_value > THRESHOLD:
        baichuan_copy_to_compress = tmp_merged_model
        lay -= INTERVAL
        if lay >= len(baichuan_copy_to_compress.model.layers):
            lay = len(baichuan_copy_to_compress.model.layers) - 1 - MERGE_LAYERS
    else:
        lay -= 1

28
current model layer 32
Similarities: [0.7035664319992065, 0.7780175805091858] Mean similarity: 0.7407920062541962
26
current model layer 29
Similarities: [0.6775448322296143, 0.7487741708755493] Mean similarity: 0.7131595015525818
24
current model layer 27
Similarities: [0.6815054416656494, 0.7251859903335571] Mean similarity: 0.7033457159996033
22
current model layer 25
Similarities: [0.6548051834106445, 0.7072852253913879] Mean similarity: 0.6810452044010162
21
current model layer 25
Similarities: [0.6220227479934692, 0.6702566146850586] Mean similarity: 0.6461396813392639
20
current model layer 25
Similarities: [0.6027112007141113, 0.6953749656677246] Mean similarity: 0.649043083190918
19
current model layer 25
Similarities: [0.6146378517150879, 0.6851768493652344] Mean similarity: 0.6499073505401611
18
current model layer 25
Similarities: [0.6497379541397095, 0.6648099422454834] Mean similarity: 0.6572739481925964
17
current model layer 25
Similarities: [0.6446682810783386, 0.70

In [10]:
baichuan_copy_to_compress.config.num_hidden_layers = len(baichuan_copy_to_compress.model.layers)
baichuan_copy_to_compress

BaiChuanForCausalLM(
  (model): Model(
    (embed_tokens): Embedding(64000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-24): 25 x DecoderLayer(
        (self_attn): Attention(
          (W_pack): Linear(in_features=4096, out_features=12288, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=64000, bias=False)
)

In [11]:
baichuan_copy_to_compress.config.num_hidden_layers

25

In [12]:
baichuan_copy_to_compress.save_pretrained("/content/pruned_model")
tokenizer.save_pretrained("/content/pruned_model")

('/content/pruned_model/tokenizer_config.json',
 '/content/pruned_model/special_tokens_map.json',
 '/content/pruned_model/tokenizer.model',
 '/content/pruned_model/added_tokens.json')

In [13]:
with open("/content/pruned_model/README.md", "w") as f:
    f.write("# Pruned Model\n\nThis is a pruned version of the Baichuan-7b model.")

In [14]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="namannn/Baichuan-7b-laco", exist_ok=True)
api.upload_folder(
    folder_path="/content/pruned_model",
    repo_id="namannn/Baichuan-7b-laco",
    repo_type="model",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:3757: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


model-00005-of-00005.safetensors:   0%|          | 0.00/2.85G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/namannn/Baichuan-7b-laco/commit/9e204a753e576bcb2a8e9bab52031294fdcfa3ad', commit_message='Upload folder using huggingface_hub', commit_description='', oid='9e204a753e576bcb2a8e9bab52031294fdcfa3ad', pr_url=None, pr_revision=None, pr_num=None)